In [9]:
import numpy as np
from astropy import constants as cst
from astropy import units as units
import sys
sys.path.insert(1, '/lhome/silkem/ChemTorch/ChemTorch/src')
sys.path.insert(1, '/lhome/silkem/ChemTorch/ChemTorch/src/ode')
import rates as r
import odes as o

### Physical constants

In [40]:
## Physical constants
kB = cst.k_B.cgs.value          ## Boltzmann constant [erg/K]
mH = cst.u.to('g').value        ## mass H atom = atomic mass unit [g]
Msun = cst.M_sun.cgs.value      ## gram
yr = units.year.to('s')         ## year in seconds
Msunyr = Msun/yr                ## units of mass-loss rate in gram/s
cms  = 1e5                      ## units of velocity in cm/s
g_to_kg = units.gram.to('kg')
cm_to_m = units.cm.to('m')

## Other constants
pi = np.pi
mu = 2.0 + 4.0*0.17             ## mu (average mass per H2 molecule), taking into account the abundance of He


#-- GRAIN PARAMETERS FOR H2 FORMATION AND CR IONISATION
rGr = 1.0E-5        ## grain radius [cm]
nGr = 1.5e-12       ## grain number density/H2 (assuming gas/dust = 200, rho = 3.5 g/cm^3)
w = 0.5             ## grain albedo
AUV_AV = 4.65
stckH = 0.3         ## sticking coefficient for H atoms

In [39]:
print(meter)

0.01


In [38]:
'''
TODO 
om te vermijden dat er kei veel moet geloopt worden over elke lijst van elke input parameter, 
kan er gesplit worden gewoon op spatie, en dan een ander keyword meegeven om te beslissen of alle
params gecombineerd worden, of dat het per model is.

'''



infile = '/lhome/silkem/ChemTorch/ChemTorch/input/input.txt'

filelines = list()

with open(infile, 'r') as f:
    lines = f.readlines()
    

print(lines)


for i in range(len(lines)):
    lines[i]=lines[i].replace(' \n', '')
    lines[i]=lines[i].replace('\n', '')
    lines[i]=lines[i].split(' ')

    if lines[i][0] == 'dens':
        dens_idx = i
        dens_line = lines[i][2:]

    if lines[i][0] == 'temp':
        temp_idx = i
        temp_line = lines[i][2:]

    if lines[i][0] == 'delta':
        delta_idx = i
        delta_line = lines[i][2:]

    if lines[i][0] == 'Av':
        Av_idx = i
        Av_line = lines[i][2:]


print(dens_line)
print(temp_line)

if len(dens_line) > 1 and dens_line[1] == ':':
    type = 'single'

if len(dens_line) > 1 and dens_line[1] == ';':
    type = 'combine'

else: type = 'none'

print(type)

input = dict()

['### Inputfile TorchChem ###\n', '### ------------------- ###\n', '\n', '## Use : to split input to create specific models.\n', '## Use ; to cross all input values.\n', '\n', '\n', '## --\n', '\n', '# density [Msol/yr]\n', 'dens = 1e-6 ; 1e-5\n', '\n', '# temperature [K]\n', 'temp = 2000\n', '\n', '# overall dilution of the radiation field\n', 'delta = 1\n', '\n', '# species-specific extinction (connected to optical depth)\n', 'Av = 1\n', '\n', '## --']
['1e-6', ';', '1e-5']
['2000']
combine


### Three body reactions

In [3]:
def read_rate_file():

    loc = 'rates/rate16_IP_2330K_AP_6000K.rates'

    rates = dict()
    with open(loc, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            line = lines[i].split(':')
            rates[int(line[0])] = line[1:]
    
    type = list()
    no = list()
    α = np.zeros(len(rates))
    β = np.zeros(len(rates))
    γ = np.zeros(len(rates))
    for nb in rates:
        no.append(nb)
        type.append(str(rates[nb][0]))
        α[nb-1] = float(rates[nb][8])
        β[nb-1] = float(rates[nb][9])
        γ[nb-1] = float(rates[nb][10])

    return rates, no, type, α, β, γ

In [104]:
# Read the rates file and return the rate coefficients per reaction: 
#               reaction no.:type:R1:R2:P1:P2:P3:P4:NE:α : β : γ: Tl :Tu:ST:ACC:REF
# indices                 0    1  2  3  4  5  6  7  8  9  10  11  12  13 14 15 

In [63]:
rates, no, type, α, β, γ = read_rate_file()

In [4]:
def read_specs_file(chemtype):

    loc = 'rates/rate16_IP_6000K_'+chemtype+'rich_mean_Htot.specs'

    # specs = dict()

    specs = np.loadtxt(loc, skiprows=1,   max_rows=466, usecols=(1), dtype=str)
    consv = np.loadtxt(loc, skiprows=468, max_rows=2  , usecols=(1), dtype=str)
    parnt = np.loadtxt(loc, skiprows=471   , usecols= (0,1), dtype=str)
    

    return specs, (parnt).T, consv

In [38]:
specs, parnt, consv = read_specs_file('C')

### Calculating rates

In [5]:
## input values physics
ρ = 1e-6
T = 2500.
δ = 1.
Av = 1.

## input chemistry
chemtype = 'C'

## calculate H accretion on dust
Haccr = stckH *pi*(rGr**2.0)*ρ*nGr*(8.0*kB*T/(pi*mH))**0.5

In [11]:
def calculating_rates(T, δ, Av):

    rates,type, α, β, γ = read_rate_file()

    print(len(type))

    k = np.zeros(len(type)+1)

    for i in range(len(type)):
        if type[i] == 'CP':
            k[i+1] = r.CP_rate(α[i]) 
        if type[i] == 'CR':
            k[i+1] = r.CR_rate(α[i], β[i], γ[i], T)
        if type[i] == 'PH':
            k[i+1] = r.photodissociation_rate(α[i], γ[i], δ, Av)
        else:
            k[i+1] = r.Arrhenius_rate(α[i], β[i], γ[i], T)

    return k


def initialise_abs(chemtype):
    specs, parnt, consv = r.read_specs_file(chemtype)

    ## Initial abundances of the non-conserved species
    abs = np.zeros(len(specs)+1)

    for i in range(len(specs)):
        for j in range(len(parnt)):
            if specs[i] == parnt[0][j]:
                abs[i+1] = parnt[1][j]
        # if specs[i] == 'CO'
        #     iCO = i

    ## Initialise abundances of the conserved species
    abs_consv = np.zeros(len(consv)+1)
    abs_consv[2] = 0.5

    return abs, abs_consv

In [12]:
n, n_consv = initialise_abs(chemtype)

In [8]:
ndot = np.zeros(len(n))
X    = np.zeros(len(n_consv))

k = calculating_rates(T, δ, Av)

6852


In [9]:
X, ndot = o.ODE(n, n_consv, ndot, X, k, ρ, Haccr)

### Outflow density

In [51]:
def density(Mdot,v, r):
    '''
    Input 
        - mass-loss rate (Mdot) in units of Msol/yr
        - outflow velocity (v) in units of km/s
        - radius (r): location of the outflow, inputs of cm
    Output
        - density in units of g/cm^3
    '''
    # r    = 1e18 #* unt.cgs.cm                       # cm
    Mdot = Mdot * Msunyr                            # gram/s
    v    = v    * cms                               # cm/s


    dens = Mdot / (4*pi * v * r**2 * mu * mH)       # g/cm^3

    # dens = dens * g_to_kg * cm_to_m**(-3)           # kg/m^3

    return dens

In [52]:
r = 1.e15
print('{:.2E}'.format(density(1.e-5, 10., r)))
print('{:.2E}'.format(density(1.e-6, 15., r)))
print('{:.2E}'.format(density(1.e-7, 10., r)))
print('{:.2E}'.format(density(1.e-8, 5. , r)))

1.13E+07
7.51E+05
1.13E+05
2.25E+04
